# CS445: Computational Photography
## Programming Project 4: Image-Based Lighting


## Recovering HDR Radiance Maps 

Load libraries and data

In [1]:
# jupyter extension that allows reloading functions from imports without clearing kernel :D
%load_ext autoreload
%autoreload 2

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# System imports
import os
from os import path
import math

# Third-Party Imports
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.interpolate import griddata

%matplotlib inline
from random import random
import time
import scipy
import scipy.sparse.linalg

from IPython.display import HTML
from IPython.display import Video
import shutil

# modify to where you store your project data including utils
datadir = "C:/Users/chaob/Desktop/445/finalproject/fp/FinalProject" 

utilfn = datadir + "utils"
# !cp -r "$utilfn" .
samplesfn = datadir + "samples"
# !cp -r "$samplesfn" .

# can change this to your output directory of choice
# !mkdir "images"
# !mkdir "images/outputs"

# import starter code
import utils
from utils.io import read_image, write_image, read_hdr_image, write_hdr_image
from utils.display import display_images_linear_rescale, rescale_images_linear
from utils.hdr_helpers import gsolve
from utils.hdr_helpers import get_equirectangular_image
from utils.bilateral_filter import bilateral_filter
from utils.bilateral_filter import bilateral_filter



### Reading Videos



In [3]:
def import_video(video_path):

    # Create a directory to store the extracted frames
    output_folder = 'frames'+ '/' + vname
    
    # Check if the directory exists
    if os.path.exists(output_folder):
        # If it exists, remove the directory and its contents
        shutil.rmtree(output_folder)
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)

    # Get the video properties
    width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Initialize variables
    frame_count = 0
    frames = []

    # Read frames from the video
    while True:
        ret, frame = video.read()
        if not ret:
            break

        # Save the frame as an image in the output folder
        frame_path = os.path.join(output_folder, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_path, frame)

        # Append the frame to the list
        frames.append(frame)

        frame_count += 1

    # Release the video object
    video.release()

    print(f"Total frames: {total_frames}")
    print(f"Frames extracted: {frame_count}")
    print(f"Frames saved in the folder: {output_folder}")

    return frames, width, height, fps


In [4]:
imdir = 'samples'
vname = 'cat1'

# Specify the path to the video file
video_path = imdir + '/' + vname +'.mp4'

# Import the video and get the frames and video properties
frames, width, height, fps = import_video(video_path)

Total frames: 144
Frames extracted: 144
Frames saved in the folder: frames/cat1


In [5]:
# Display the video in the notebook
video = Video(video_path)
print(video_path)
display(video)

samples/cat1.mp4


In [6]:
# First frame is background, or can be frames[0]
background_image_file = 'frames'+ '/' + vname + '/' +'frame_0000.jpg'
background_image = read_image(background_image_file)

# background_image_file = imdir + '/' + 'empty.jpg'
# background_image = read_image(background_image_file)

### Background removal

Compute the HDR image as average of irradiance estimates from LDR images

In [64]:
def background_subtraction_o(frames, width, height, fps, vname):
    # Create directories to store the output frames and video
    output_folder_bs = "background_removed_frames_o" + '/' + vname
    
    # Check if the directory exists
    if os.path.exists(output_folder_bs):
        # If it exists, remove the directory and its contents
        shutil.rmtree(output_folder_bs)
    os.makedirs(output_folder_bs, exist_ok=True)
    
    # Read the first frame as the background image
    background = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    
    # Apply blurring to the background frame
    blurred_background = cv2.GaussianBlur(background, (15, 15), 0)
    
    # Initialize variables
    processed_frames = []
    
    # Create a kernel for morphological operations
    kernel = np.ones((200, 200), np.uint8)
    
    # Process frames for background subtraction
    for i, frame in enumerate(frames):
        # Convert the frame to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Apply blurring to the grayscale frame
        blurred_frame = cv2.GaussianBlur(gray_frame, (15, 15), 0)
        
        # Perform background subtraction
        diff = cv2.absdiff(blurred_background, blurred_frame)
        
        # Apply thresholding
        threshold = 60
        _, thresh = cv2.threshold(diff, threshold, 255, cv2.THRESH_BINARY)
        
        # Apply erosion
        eroded = cv2.erode(thresh, kernel, iterations=10)
        
        # # Apply dilation
        dilated = cv2.dilate(eroded, kernel, iterations=3)
        
        # Save the thresholded frame as an image in the output folder
        frame_path_bs = os.path.join(output_folder_bs, f"frame_{i:04d}.jpg")
        cv2.imwrite(frame_path_bs, thresh)
        
        # Append the processed frame to the list
        processed_frames.append(thresh)
    
    print(f"Background subtraction completed.")
    print(f"Output frames saved in the folder: {output_folder_bs}")
    
    return processed_frames, output_folder_bs

In [65]:
# Perform background subtraction on the frames and get the processed frames
processed_frames_o, output_folder_bs_o = background_subtraction_o(frames, width, height, fps, vname)

Background subtraction completed.
Output frames saved in the folder: background_removed_frames_o/cat1


In [66]:
%%capture
!ffmpeg -framerate 30 -i "{output_folder_bs_o}/frame_%04d.jpg" -c:v libx264 -pix_fmt yuv420p "{output_folder_bs_o}/bs_video.mp4"

In [ ]:
#会占用导致无法更改，最后展示用
# # Display the video in the notebook
# v="background_removed_frames/cat1/bs_video.mp4"
# video = Video(v)
# display(video)

In [131]:
def background_subtraction_improve(frames, width, height, fps, vname):
    # Create directories to store the output frames and video
    output_folder_bs = "background_removed_frames_improve" + '/' + vname
    
    # Check if the directory exists
    if os.path.exists(output_folder_bs):
        # If it exists, remove the directory and its contents
        shutil.rmtree(output_folder_bs)
    os.makedirs(output_folder_bs, exist_ok=True)
        
    # Initialize variables
    processed_frames = []
    
    # Create a kernel for morphological operations
    kernel = np.ones((80, 80), np.uint8)
    
    # Create a GMM-based background subtractor
    background_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows=True)
    
    # Define the number of initial frames to skip or use for training
    num_initial_frames = 5

    # Train the background subtractor on initial frames
    for i in range(num_initial_frames):
        frame = frames[i]
        blurred_frame = cv2.GaussianBlur(frame, (35, 35), 0)
        _ = background_subtractor.apply(blurred_frame)

    # Process frames for background subtraction
    for i, frame in enumerate(frames):
        
        blurred_frame = cv2.GaussianBlur(frame, (35, 35), 0)
        # Apply background subtraction
        mask = background_subtractor.apply(blurred_frame)

        # Apply thresholding to the mask
        _, thresh = cv2.threshold(mask, 250, 255, cv2.THRESH_BINARY)
    
       
        # Apply erosion
        eroded = cv2.erode(thresh, kernel, iterations=2)
        
        # # Apply dilation
        dilated = cv2.dilate(eroded, kernel, iterations=10)
        
        # Save the thresholded frame as an image in the output folder
        frame_path_bs = os.path.join(output_folder_bs, f"frame_{i:04d}.jpg")
        cv2.imwrite(frame_path_bs, thresh)
        
        # Append the processed frame to the list
        processed_frames.append(thresh)
    
    print(f"Background subtraction completed.")
    print(f"Output frames saved in the folder: {output_folder_bs}")
    
    return processed_frames, output_folder_bs

In [132]:
# Perform background subtraction on the frames and get the processed frames
processed_frames_improve, output_folder_bs_improve = background_subtraction_improve(frames, width, height, fps, vname)

Background subtraction completed.
Output frames saved in the folder: background_removed_frames_improve/cat1


In [133]:
%%capture
!ffmpeg -framerate 30 -i "{output_folder_bs_improve}/frame_%04d.jpg" -c:v libx264 -pix_fmt yuv420p "{output_folder_bs_improve}/bs_video.mp4"

In [227]:
def filter_white_regions(image_path):
    # Read the image in grayscale
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    # Find contours in the binary image
    contours, _ = cv2.findContours(image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Check if any contours were found
    if len(contours) > 0:
        # Find the area of the largest contour
        largest_area = max(cv2.contourArea(contour) for contour in contours)
        
        # Set the area threshold to be the area of the largest contour minus 200
        area_threshold = largest_area - 0.05*largest_area
        
        # Create a mask to store the filtered white regions
        mask = np.zeros_like(image)
        
        # Iterate over each contour
        for contour in contours:
            # Calculate the area of the contour
            area = cv2.contourArea(contour)
            
            # If the area is greater than or equal to the threshold, draw the contour on the mask
            if area > 2000 and area >= area_threshold:
                cv2.drawContours(mask, [contour], 0, 255, -1)
        
        # Apply the mask to the original image
        result = cv2.bitwise_and(image, image, mask=mask)
    else:
        # If no contours were found, return the original image
        result = image
    
    return result

In [228]:

# Specify the folder path containing the black and white images
folder_path = output_folder_bs_improve

# Create a new folder to store the processed images
out_folder = "obj_frames_improve" + '/' + vname
# Check if the directory exists
if os.path.exists(out_folder):
    # If it exists, remove the directory and its contents
    shutil.rmtree(out_folder)
os.makedirs(out_folder, exist_ok=True)


# Iterate over each image file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") :  # Adjust the file extensions as needed
        image_path = os.path.join(folder_path, filename)
        
        # Process the image to keep the white regions with areas greater than or equal to the threshold
        processed_image = filter_white_regions(image_path)
        
        # Save the processed image to the output folder
        output_path = os.path.join(out_folder, filename)
        cv2.imwrite(output_path, processed_image)

In [229]:
%%capture
!ffmpeg -framerate 30 -i "{out_folder}/frame_%04d.jpg" -c:v libx264 -pix_fmt yuv420p "{out_folder}/obj_video.mp4"